# Example code

## Importing library

In [1]:
import sys,os
homelib = ".."
if not homelib in sys.path:
    sys.path.append(homelib)

from object_publisher import *

## Defining class to be published in CLI and Flask web app
`publish` decorator is specified for method to be used as sub-command or URL handler.

in CLI, every method is called with following arugment.

```
$ exe <methodname> [parameters ...]
```

method arguments without default value is used as positional parameter, and arguments with default value is used as optional parameter (`--<arg name>`)


in Flask App, method is called with follwing URL access.

```
/<classname>/[path spec]/<methodname>
```
where path spec is specified in the decorator:
```
@publish(flask={"path": ["args to be specified in path", ...] })
```

source of the request parameter is specified by:
```
@publish(flask={"params": "<type>"})
```
where `<type>` is one of:

|value|notes|
|-|-|
|args|parameters are specified as field parameters in the URL.|
|form|parameters are specified as content body of the requests, with standard form representation.|
|json|parameters are specified as content body of the requests, with 'Content-type: application/json'|

### Example class
`hello`, `world`, `test`, `show_resource` is defined to be published.
`not_published` is not published, so that no interface is not build for `not_published` method.

docstring is used as source of help in the CLI interface.

In [2]:
class Test:
    
    def __init__(self):
        self.x = "X"
        self.y = "Y"
    
    @publish
    def hello(self):
        return "Hello"
    
    def not_published(self):
        return "private.: %s"%self.x
    
    @publish 
    def world(self, name):
        """
        Worldコマンド
        
        Parameters
        ----------
        name: str
            名前（表示用）
        
        Returns
        -------
        """
        return name
    
    @publish(flask = {"params": "form"})
    def test(self, name, kw=None):
        """
        Testコマンド
        
        Little bit detailed description of Test method.
        
        Parameters
        ----------
        name: str
            名前（表示用）
        kw: str
            表示用のキーワード
        
        Returns
        -------
        """
        return "%s-%s-%s"%(self.y, name, kw)
    
    @publish(flask = {"path": ["id", "attr"], "params": "json"})
    def show_resource(self, id, attr, mustarg, someargs="test"):
        print("TestID: %s"%id)
        print("TestAttr: %s"%attr)
        print("MustArg: %s"%mustarg)
        print("SomeArgs: %s"%someargs)
        
        return {"TestID": id, "TestAttr": attr, "MustArg": mustarg, "SomeArgs": someargs}

# Typical usage

## First step: Creating object, and call method as usual library instance.
published method can be used as normal method. No special handling is required to invoke published method.

In [3]:
obj = Test()
display(obj.hello())
display(obj.not_published())
display(obj.world("me"))
display(obj.show_resource("id1", "attr1", "must"))

'Hello'

'private.: X'

'me'

TestID: id1
TestAttr: attr1
MustArg: must
SomeArgs: test


{'TestID': 'id1', 'TestAttr': 'attr1', 'MustArg': 'must', 'SomeArgs': 'test'}

## Second step: Call object from command line interface
Buliding CLI interface object, and run it wtih sys.argv or parameter array.
You can build interface by one of two initialization parameters:

1. specifying instance by `object=<object>`

    Same object is used throughout several `run` calls in the same process.

2. specifying class (and optional allocator/deallocator) by `klass=<class>, [allocator=<object allocation function>, deallocator=<object deallocation function>]`

    New object is created for every call of the `run` method.
    If allocator / deallocator is not specified, simple factory to call default <class> constructor is automatically generated. 

### Global help

In [4]:
try:
    CLI(object=obj).run(["-h"])
except SystemExit as e:
    pass

usage: Test [-h] {hello,world,test,show_resource} ...

positional arguments:
  {hello,world,test,show_resource}
                        sub-command help
    hello
    world               Worldコマンド
    test                Testコマンド
    show_resource

optional arguments:
  -h, --help            show this help message and exit


### Sub-command help for 'hello' method.

In [5]:
try:
    CLI(klass=Test).run(["hello", "-h"])
except SystemExit as e:
    pass

usage: Test hello [-h]

optional arguments:
  -h, --help  show this help message and exit


### Run method by "hello" subcommand.

In [6]:
CLI(klass=Test).run(["hello"])

'Hello'


### Sub-command help for 'world' method.

docstring is used as the source of the parameter description.

In [7]:
try:
    CLI(klass=Test).run(["world", "-h"])
except SystemExit as e:
    pass

usage: Test world [-h] <name>

positional arguments:
  <name>      名前（表示用）

optional arguments:
  -h, --help  show this help message and exit


### Run method by "world" subcommand.
`name` parameter is filled with first sub-command argument, because `name` is positional argument. 

In [8]:
CLI(object=obj).run(["world", "him"])

'him'


### Sub-command help for 'test' method.
`name` is treated as positional parameter because it has no default value,
while `kw` is treated as optional parameter because it has default value (`None`)

In [9]:
try:
    CLI(object=obj).run(["test", "-h"])
except SystemExit as e:
    pass

usage: Test test [-h] [--kw <kw>] <name>

positional arguments:
  <name>      名前（表示用）

optional arguments:
  -h, --help  show this help message and exit
  --kw <kw>   表示用のキーワード


### Run method by "test" subcommand without any optional parameters.

In [10]:
CLI(object=obj).run(["test", "its"])

'Y-its-None'


### Run method by "test" subcommand with explicit optional parameters.

In [11]:
CLI(klass=Test).run(["test", "its", "--kw", "TEST"])

'Y-its-TEST'


### Sub-command help for 'show_resource' method.
This method has explicit annotation for flask mode, so that parameter of CLI is treated different way from web app.

In [12]:
try:
    CLI(klass=Test).run(["show_resource", "-h"])
except SystemExit as e:
    pass

usage: Test show_resource [-h] [--someargs <someargs>] <id> <attr> <mustarg>

positional arguments:
  <id>
  <attr>
  <mustarg>

optional arguments:
  -h, --help            show this help message and exit
  --someargs <someargs>


### Run method by "show_resource" subcommand.

In [13]:
CLI(klass=Test).run(["show_resource", "test","ATTR","must!!!","--someargs","keyword"])

TestID: test
TestAttr: ATTR
MustArg: must!!!
SomeArgs: keyword
{'MustArg': 'must!!!',
 'SomeArgs': 'keyword',
 'TestAttr': 'ATTR',
 'TestID': 'test'}


## Third step: Publish object as web service in Flask.

**CAUTION**: After testing, you need to stop notebook manually.

after launching web service, you can try it as follwing command in the terminal.

1. Simple "GET" service.

```
$ curl http://localhost:9999/Test/hello
```

2. "GET" service with arguments

```
$ curl http://localhost:9999/Test/world?name=test
```

3. "POST" service with form representation for arguments

      `@publish(flask={"params": "form"})` is used.
      method is automatically guessed as "POST" because it has "form" type parameter.
      You can override it by explicitly specifying method in the decrorator.

```
$ curl -X POST --form name="Example user" http://localhost:9999/Test/test
```

4. "POST" service with JSON representation for arguments

      "`id`" and "`attr`" is specified as part of path in URL.
      `@publish(flask={"path": ["id", "attr"], "params": "json", "method": "POST"})` is used.

```
$ curl -X POST -d '{"mustarg": 1}' -H 'Content-type: application/json' http://localhost:9999/Test/trend/world/show_resource
```

In [17]:
Flask(klass=Test).run(port=9999)

 * Serving Flask app "Test" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:9999/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Oct/2020 01:06:32] "GET /Test/hello HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2020 01:07:31] "GET /Test/world?name=test HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2020 01:08:10] "POST /Test/test HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2020 01:08:30] "POST /Test/trend/world/show_resource HTTP/1.1" 200 -


TestID: trend
TestAttr: world
MustArg: 1
SomeArgs: test
